In [2]:
import pandas as pd

# 데이터 준비
## csv 파일을 데이터 프레임으로 변환한 후 
## head() 메서드로 처음 5개 행 출력
fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()

## 어떤 종류의 생선이 있는지 unique() 함수를 통해 Species 열에서 고유한 값 추출
print(pd.unique(fish['Species']))


## Species 열을 타깃으로 만들고 나머지 5개 열을 입력 데이터로 사용
fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()

print(fish_input[:5])

## 동일한 방식으로 타겟 데이터 생성
fish_target = fish['Species'].to_numpy()

## 데이터를 훈련 세트와 테스트 세트로 나눈다
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target, random_state=42)

## 사이킷 런의 StandardScaler 클래스를 사용해 훈련 세트와 테스트 세트를 표준화 전처리
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)




# k-최근접 이웃 분류기의 확률 예측
## KNeighborsClassifier 클래스 객체로 훈련
## 최근접 이웃 개수 k = 3
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)

print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))


# 다중분류
## 위와 같이 이진 분류와 모델을 만들고 훈련하는 방식은 동일하다
## 이진 분류 - 양성 클래스와 음성 클래스를 1과 0으로 지정하여 타깃 데이터 생성
## 다중 분류 - 다중 분류도 타깃값을 숫자로 바꾸어 입력할 수 있지만, 
##            사이킷런에서는 문자열로 된 타깃값을 그대로 사용 가능
### 주의할 점 : 타깃값을 그대로 사이킷런 모델에 전달하면 순서가 자동으로 알파벳 순으로 정렬됨
###           print(kn.classes_) 를 통해 순서를 확인할 수 있다.

print(kn.classes_)

## 테스트 세트에 있는 처음 5개 샘플의 타깃값 예측
print(kn.predict(test_scaled[:5]))

## 테스트 세트에 있는 처음 5개 샘플에 대한 확률 출력
## 넘파이 round() 함수 : 기본적으로 소수점 첫째 자리에서 반올림. decimals 매개변수로 유지할 수수점 아래 자릿수 지정 가능
import numpy as np
proba = kn.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=4))




ModuleNotFoundError: No module named 'pandas'

In [ ]:
# 로지스틱 회귀

import numpy as np
import matplotlib.pyplot as plt
z = np.arange(-5, 5, 0.1)
phi = 1/(1+ np.exp(-z))
plt.plot(z, phi)
plt.xlabel('z')
plt.ylabel('phi')
plt.show()


In [ ]:
# 로지스틱 회귀로 이진 분류 수행

## 불리언 인덱싱 : 넘파이 배열에 True, False 값을 전달하여 행을 선택하는 것
### 'A'와 'C'만 골라내기위해 첫 번째와 세 번째 원소에 True 값을 전달하고 나머지는 False 전달
char_arr = np.array(['A', 'B', 'C', 'D', 'E'])
print(char_arr[[True, False, True, False, False]])


In [ ]:
## 위와 같은 방식으로 훈련 세트에서 도미와 빙어의 행만 골라내자
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]

## 현재 bream_smelt_indexes 배열은 도미와 빙어일 경우 True 그 외는 모두 False
## 이 배열을 사용해 train_scaled와 train_target 배열에 불리언 인덱싱을 적용하면 
## 도미와 빙어 데이터만 골라낼 수 있다.

## 위 데이터로 로지스틱 회귀 모델을 훈련
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)

## train_bream_smelt에 있는 처음 샘플 5개 예측
print(lr.predict(train_bream_smelt[:5]))

## predict_proba() 메서드로 train_bream_smelt 에서 처음 5개의 샘플의 예측 확률 출력
print(lr.predict_proba(train_bream_smelt[:5]))

print(lr.classes_)



In [ ]:
## 로지스틱 회귀가 학습한 계수 확인
print(lr.coef_, lr.intercept_)

In [ ]:
# z값을 출력해주는 decisions 함수 
decisions = lr.decision_function(train_bream_smelt[:5])
print(decisions)

In [ ]:
# z 값을 시그모이드 함수에 통과시키면 함수를 얻을 수 있다.
## 파이썬의 사이파이 라이브러리의 시그모이드 함수인 expit() 를 사용해보자
from scipy.special import expit
print(expit(decisions))